In [45]:
import tensorflow as tf
import pandas as pd

# Tried using Textblob for semantic analysis : Too Intensive
from textblob import TextBlob

# For encoding and vectorization we use scikit
from sklearn import preprocessing, model_selection
from sklearn.feature_extraction.text import TfidfVectorizer

#For shallow neural network
from keras import layers,models


In [46]:
train_data = pd.read_csv('/Users/ayush/train.csv')

# Semanitc Analysis : Takes too long
# 
# copy_article = train_data["text"]
# 
# for i in range(len(copy_article)):
#     semantic = TextBlob(copy_article[i])
#     semantic = semantic.sentiment
#     train_data["text"][i] = semantic

train_x = train_data["text"]
train_y = train_data["label"]

# Splitting into training and testing
train_x,test_x,train_y,test_y = model_selection.train_test_split(train_x,train_y)


encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)



In [44]:
# ngram level tf-idf ( For word to vector)
# code snippet used
tfidf_vector_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vector_ngram.fit(train_data["text"].values.astype('U'))
xtrain_tfidf_ngram =  tfidf_vector_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vector_ngram.transform(train_x)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [41]:
# Basic model for testing if word to vector works
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 